In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

loader = UnstructuredFileLoader("./document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 주어진 문서를 읽고 해석합니다. 주어진 문서를 사용하여 질문에 상세하게 대답하십시오. 답을 모르면 지어내지 말고 모른다고 대답하십시오.\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


def load_memory():
    return memory.load_memory_variables({}).get("chat_history", "")


def invoke_chain(question):
    chat_history = memory.load_memory_variables({}).get("chat_history", "")

    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    inputs = {
        "context": context,
        "question": question,
        "chat_history": chat_history,
    }

    prompt = prompt_template.format_messages(**inputs)
    response = llm(prompt)

    memory.save_context({"question": question}, {"answer": response.content})

    return print(response.content)


invoke_chain("Aaronson 은 유죄인가요?")


주어진 문서에서는 Aaronson에 대한 직접적인 언급이 없기 때문에 그의 유죄 여부에 대한 정보를 제공할 수 없습니다. Aaronson이 누구인지, 어떤 사건과 관련이 있는지에 대한 추가적인 정보가 필요합니다.


In [19]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")


그는 테이블에 다음과 같은 메시지를 썼습니다:

1. FREEDOM IS SLAVERY
2. TWO AND TWO MAKE FIVE
3. GOD IS POWER

이 메시지들은 그의 생각을 나타내며, 그는 이러한 문구들을 쓰면서 자신의 사고를 재교육하려고 했습니다.


In [20]:
invoke_chain("Julia는 누구인가요?")


Julia는 조지 오웰의 소설 "1984"에서 주인공인 윈스턴 스미스의 연인입니다. 그녀는 당의 억압적인 체제에 반대하는 감정과 행동을 가지고 있으며, 윈스턴과 함께 금지된 사랑을 나누고 반체제적인 생각을 공유합니다. 그러나 소설의 진행 과정에서 그들의 관계는 당의 감시와 억압에 의해 위협받고, 결국 그들은 서로에게 배신하게 됩니다. Julia는 윈스턴에게 중요한 인물로, 그들의 사랑은 자유와 개인의 감정을 상징하는 요소로 작용합니다.
